In [84]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt

import time

from functools import partial
#from torchsummary import summary

In [85]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
 
#partial function, sets parameters, kernel_size and bias by default. When used conv3x3 those parameters will be ued
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)    

In [86]:
#in this example, even that kernel_size and bias are not used, those values are taken from the definition in partial function
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [87]:
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [88]:
#This function is intented to be extended by other class
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [89]:
#Just test Residualblock, the output should be 2
dummy = torch.ones(2,2)
block = ResidualBlock(64, 64)
block(dummy)

tensor([[2., 2.],
        [2., 2.]])

In [90]:
#shortcut applies a convolution of 1 kernel to transform in_channels to out_channels

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [91]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs), nn.BatchNorm2d(out_channels))

In [92]:
class ResNetBasicBlock(ResNetResidualBlock):
    """
    Basic ResNet block composed by two layers of 3x3conv/batchnorm/activation
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    

In [93]:
dummy = torch.ones((1, 32, 224, 224))

block = ResNetBasicBlock(32, 64)
block(dummy).shape
print(block)

ResNetBasicBlock(
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (activate): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [94]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )

In [95]:
#example below uses list comprehension

class ResNetLayer(nn.Module):
    """
    A ResNet layer composed by `n` blocks stacked one after the other
    """
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            #This is a list comprehension pased to the Sequential function using * argument
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [96]:
dummy = torch.ones((1, 64, 48, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
layer(dummy).shape

torch.Size([1, 128, 24, 24])

In [97]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation='relu', block=ResNetBasicBlock, *args, **kwargs):
        super().__init__()
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation_func(activation),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        #zip, creates a tuple conbining both list, if one list is smaller result will take its lenght
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,*args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

In [98]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

In [99]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [107]:
def resnet18(in_channels, n_classes, block=ResNetBasicBlock, *args, **kwargs):
    return ResNet(in_channels, n_classes, block=block, deepths=[2, 2, 2, 2], *args, **kwargs)

### Test model

In [151]:
#hyperparameters
n_epochs = 2
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.001
momentum = 0.5
log_interval = 10

In [152]:
data_dir = 'data/'

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                                 ])),
  batch_size=batch_size_test, shuffle=True)
    
type(test_loader)

torch.utils.data.dataloader.DataLoader

In [153]:
#Resnet model
model = resnet18(1, 10)

#Stochastic gradient decent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)
#Criterion
loss_f = nn.CrossEntropyLoss()

In [154]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [155]:
def train(epoch):
  #
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    
    #set parameters gradients to 0
    optimizer.zero_grad()
    
    #runs model with data and returns result in output
    #here data contains a 64 * 1 * 28 * 28 tensor, 64 is batch size
    output = model(data)
    
    loss = loss_f(output, target)
    
    #with this gradients are calculated
    loss.backward()
    
    #update gradients
    optimizer.step()
    
    #Display iteration statistics
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      #torch.save(network.state_dict(), 'results/model.pth')
      #torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [158]:
def test():
  #Sets the module in evaluation mode
  model.eval()
  test_loss = 0
  correct = 0
    
  #dont update dynamic computation graph
  with torch.no_grad():
    #for every example in test
    for data, target in test_loader:
      #evaluate the model
      output = model(data)
      #acumulate the loss
      test_loss += loss_f(output, target).item()
      
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [159]:
#test
#n_epochs = 1
#Check accuaracy before training
test()
tic = time.perf_counter()

for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
    
toc = time.perf_counter()

print(f"Time elapsed: {toc - tic:0.4f} seconds")


Test set: Avg. loss: 0.0023, Accuracy: 995/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.552624
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.394929
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.778780
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.572034
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.453360
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.349396
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.405889
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.341048
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.264438
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.167402
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.158046
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.363899
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.282624
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.326158
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.324889
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.158284
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.245698
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.174442
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.205799
Train Epoch: 1 [12160/

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.009476
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.016222
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.147067
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.067925
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.055920
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.021804
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.045934
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.015307
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.021421
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.049243
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.027736
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.144448
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.049044
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.046801
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.140901
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.025433
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.009941
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.096446
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.143761
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.064473


###  Source

* https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278

### ToDo

*  for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]. What is zip?